## Building three machine learning models including SVM, NB, and RF using sk-learn Python library to train a model to classify each tweet into positive or negative sentiment

In [8]:
# import libraries 
from google.colab import files
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import numpy as np
from scipy.sparse import hstack
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [1]:
# Upload the CSV file
uploaded = files.upload()

Saving Tweets.csv to Tweets.csv


In [2]:
data = pd.read_csv("Tweets.csv")

In [3]:
data = data[['text', 'airline_sentiment']]
data.head()

,text,airline_sentiment
0,@VirginAmerica What @dhepburn said.,neutral
1,@VirginAmerica plus you've added commercials t...,positive
2,@VirginAmerica I didn't today... Must mean I n...,neutral
3,@VirginAmerica it's really aggressive to blast...,negative
4,@VirginAmerica and it's a really big bad thing...,negative


In [4]:
#excluding neutral tweets to create 'two-class' classification task
data = data[data['airline_sentiment'] != 'neutral']
data.head()

,text,airline_sentiment
1,@VirginAmerica plus you've added commercials t...,positive
3,@VirginAmerica it's really aggressive to blast...,negative
4,@VirginAmerica and it's a really big bad thing...,negative
5,@VirginAmerica seriously would pay $30 a fligh...,negative
6,"@VirginAmerica yes, nearly every time I fly VX...",positive


Pre-processing tweets 

In [7]:
# noisy removal  
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

In [9]:
#stopwords removal  
stop = set (stopwords.words ("english"))
def remove_stopwords (text): 
  text = [word.lower () for word in text.split() if word.lower() not in stop]
  return " ".join(text)

In [10]:
data["text"] = data["text"].map(remove_stopwords)

In [13]:
#words vectorization using tf-idf
vectorizer = TfidfVectorizer (ngram_range=(1,2), max_features=2000, min_df=5, max_df=0.8)
tfidf = vectorizer.fit_transform(data.text)

In [14]:
#words vectorization usig BoW
count_vectorizer = CountVectorizer(ngram_range=(1,2)) 
vectorized_data = count_vectorizer.fit_transform(data.text)
indexed_data = hstack((np.array(range(0,vectorized_data.shape[0]))[:,None], vectorized_data))

In [ ]:
#print(indexed_data)

In [ ]:
#tfidf = TfidfVectorizer(ngram_range=(1,2))

In [15]:
def sentiment2target(sentiment):
    return {
        'negative': 0,
        'positive' : 1
    }[sentiment]
targets = data.airline_sentiment.apply(sentiment2target)

In [16]:
data_train, data_test, targets_train, targets_test = train_test_split(tfidf, targets, test_size=0.4, random_state=0) #call tfidf feautres vectores as for BoW we can call 'indexed_data' obj
data_train_index = data_train[:,0]
data_train = data_train[:,1:]
data_test_index = data_test[:,0]
data_test = data_test[:,1:]

# SVM Model 

In [17]:
clf = OneVsRestClassifier(svm.SVC(gamma=0.01, C=1, probability=True, class_weight='balanced', kernel='rbf'))
clf_output = clf.fit(data_train, targets_train)

In [18]:
clf_output.score(data_test, targets_test)

0.8977691141433831

In [19]:
precision_score(targets_test, clf.predict(data_test), average='weighted')

0.894158494050347

In [20]:
f1_score(targets_test, clf.predict(data_test), average='weighted')

0.8918639348032156

In [21]:
recall_score(targets_test, clf.predict(data_test), average='weighted')

0.8977691141433831

In [22]:
print(confusion_matrix(targets_test, clf.predict(data_test)))
print(classification_report(targets_test, clf.predict(data_test)))

[[3550  113]
 [ 359  595]]
              precision    recall  f1-score   support

           0       0.91      0.97      0.94      3663
           1       0.84      0.62      0.72       954

    accuracy                           0.90      4617
   macro avg       0.87      0.80      0.83      4617
weighted avg       0.89      0.90      0.89      4617



# NB Model

In [23]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
NB = MultinomialNB ()
NB_output = NB.fit(data_train, targets_train)

In [24]:
models_NB = []
models_NB.append(MultinomialNB())

for model in models_NB:
    model.fit(data_train, targets_train)
    pred = model.predict(data_test)
    acc = accuracy_score(targets_test, pred)
    print(f"Model {model.__class__.__name__} accuracy on test dataset >> {acc}")

Model MultinomialNB accuracy on test dataset >> 0.8964695689841888


In [25]:
precision_score(targets_test, NB_output.predict(data_test), average='weighted')

0.8976187713827236

In [26]:
recall_score(targets_test, NB_output.predict(data_test), average='weighted')

0.8964695689841888

In [27]:
accuracy_score(targets_test, NB_output.predict(data_test))

0.8964695689841888

In [28]:
f1_score(targets_test, NB_output.predict(data_test), average='weighted')

0.8864341955544516

# RF MODEL

In [29]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=10, random_state=0, max_features= 'sqrt' , min_samples_split= 4) 
rf_output = rf.fit(data_train, targets_train)

In [30]:
rf.score(data_test, targets_test)

0.8871561620099632

In [31]:
f1_score(targets_test, rf.predict(data_test), average='weighted')

0.8810467800038588

In [32]:
precision_score(targets_test, rf.predict(data_test), average='weighted')

0.881960383443896

In [33]:
recall_score(targets_test, rf.predict(data_test), average='weighted')

0.8871561620099632